Smart Customer Support Agent

In [ ]:
import os
from typing import TypedDict, Annotated, List
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables.graph import MermaidDrawMethod
from IPython.display import display, Image
from dotenv import load_dotenv

In [ ]:
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.2)

Defining Agent State and Prompts

In [ ]:
class SupportAgentState(TypedDict):
    messages: Annotated[List[HumanMessage | AIMessage], "The messages in the conversation"]
    user_query: str
    query_category: str
    solution: str
    statisfaction_rating: int
    follow_up_needed: bool
    support_history: List[str]

query_analysis_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a customer support query analyzer. Analyze the customer query and categorize it into ONE of these categories:

Categories:
- technical: Software bugs, login issues, app crashes, feature problems, connectivity issues
- billing: Payment problems, subscription issues, refunds, charges, account billing
- general: Basic questions, account information, how-to guides, product information
- escalate: Complex issues, angry customers, legal matters, or anything requiring human intervention

Return ONLY the category name (technical/billing/general/escalate) without explanation."""),
    ("human", "Customer Query: {query}"),
])

technical_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a technical support specialist. Provide clear, step-by-step solutions for technical issues.

Format your response as:
**Problem Diagnosed:** [Brief description]
**Solution Steps:**
1. [Step 1]
2. [Step 2]
3. [Step 3]
**Additional Help:** [Any extra tips or resources]

Keep solutions practical and easy to follow."""),
    ("human", "Technical Issue: {query}"),
])

billing_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a billing support specialist. Help customers with payment and subscription issues.

Format your response as:
**Issue Summary:** [Brief description]
**Resolution Steps:**
1. [Step 1]
2. [Step 2]
3. [Step 3]
**Next Steps:** [What happens next or follow-up needed]

Be empathetic and provide clear billing information."""),
    ("human", "Billing Issue: {query}"),
])

general_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a friendly customer service representative. Provide helpful information for general inquiries.

Format your response as:
**Answer:** [Direct answer to their question]
**Additional Information:** [Helpful related details]
**Resources:** [Where they can find more help]

Be friendly, helpful, and informative."""),
    ("human", "Customer Question: {query}"),
])